In [17]:
import os

folder_path = r"C:\Users\LEGION\Dropbox\My PC (LAPTOP-E7LP1HPD)\Desktop\Windsor\Thesis\S - 2023\MDS intro papers"
os.path.exists(folder_path)

True

In [18]:
file_list = os.listdir(folder_path)

In [19]:
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
print(os.environ["OPENAI_API_KEY"])

sk-proj-gD0Nxd5jJ5TKT3k2c2eET3BlbkFJKJzvqt3TQjt78eU4f05Z


In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
vectorstore = Chroma(embedding_function=embeddings)

for file in file_list:
    new_path = os.path.join(folder_path, file)
    loader = PyPDFLoader(new_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
    documents = text_splitter.split_documents(docs)

    vectorstore.add_documents(documents)

In [21]:
from langchain_community.llms import OpenAI
# from langchain_community.llms import Ollama

llm = OpenAI(model="gpt-3.5-turbo-instruct")
# llm = Ollama(model="llama2")
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x00000219172CAD70>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000002191502B880>, openai_api_key='sk-proj-gD0Nxd5jJ5TKT3k2c2eET3BlbkFJKJzvqt3TQjt78eU4f05Z', openai_proxy='')

In [22]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
                                          You are my Research Assistant. You will be given context via embeddings regarding a lot of research papers in my vector database.
                                          Answer the following question based only on the provided context. 
                                          Think step by step before providing a detailed answer.
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}""")

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000021916BCBBB0>)

In [25]:
from langchain.chains import create_retrieval_chain

retreival_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
retreival_chain.invoke({"input": "What are the best results within the papers you have in terms of recall and f-1 score? I need the title and author of that paper"})

{'input': 'What are the best results within the papers you have in terms of recall and f-1 score? I need the name and author of that paper',
 'context': [Document(page_content='the proposed model, whereas the existing approaches showed\nvaried results. Paper 1 obtained a 100% recall value, but the\nprecision value was very low. Paper 3 showed the highest\nresults out of the three existing approaches, with Paper 2\nnot showing satisfactory results. For attack type 8, the 2BSM\nmodel again had the best performance, with Paper 3, also\ngiving high values for both precision and recall. Overall,\nAttack types 2 and 8 are more challenging to detect, and our', metadata={'page': 9, 'source': 'C:\\Users\\LEGION\\Dropbox\\My PC (LAPTOP-E7LP1HPD)\\Desktop\\Windsor\\Thesis\\S - 2023\\MDS intro papers\\OJVT-2021-09-0079_Proof_hi.pdf'}),
  Document(page_content='recent papers. Paper 1 [14] and Paper 3 [31] performed\nsimilarly to the proposed model for attack types 1 and 4\ngenerating high precision

In [30]:
retreival_chain.invoke({"input": "Give me the list of attacks that were implemented in the VeReMi introduction paper"})

{'input': 'Give me the list of attacks that were implemented in the VeReMi introduction paper',
 'context': [Document(page_content='6 R. W. van der Heijden et al.\nmechanisms have been designed [20]). Rather than implement a broad set of at-\ntacks, we focused on this speciﬁc attack to show the eﬃcacy of our approach. We\nforesee that other researchers can contribute new attack implementations and\ncorresponding datasets to the central VeReMi repository, which we will main-\ntain. By focusing on a speciﬁc attack in this paper, we show how VeReMi is\nuseful for other researchers and provide an initial starting point for the commu-', metadata={'page': 5, 'source': 'C:\\Users\\LEGION\\Dropbox\\My PC (LAPTOP-E7LP1HPD)\\Desktop\\Windsor\\Thesis\\S - 2023\\MDS intro papers\\VeReMi_ A Dataset for Comparable Evaluation.pdf'}),
  Document(page_content='6 R. W. van der Heijden et al.\nmechanisms have been designed [20]). Rather than implement a broad set of at-\ntacks, we focused on this speciﬁc